In [3]:
import pandas as pd

## The 20 newsgroups dataset

The [20 Newsgroups data set](http://qwone.com/~jason/20Newsgroups/) is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.
The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

The data is organized into 20 different newsgroups, each corresponding to a different topic:

- 'atheism',
- 'comp.graphics',
- 'comp.os.ms-windows.misc',
- 'comp.sys.ibm.pc.hardware',
- 'comp.sys.mac.hardware',
- 'comp.windows.x',
- 'misc.forsale',
- 'rec.autos',
- 'rec.motorcycles',
- 'rec.sport.baseball',
- 'rec.sport.hockey',
- 'sci.crypt',
- 'sci.electronics',
- 'sci.med',
- 'sci.space',
- 'soc.religion.christian',
- 'talk.politics.guns',
- 'talk.politics.mideast',
- 'talk.politics.misc',
- 'talk.religion.misc']

 we will work on a partial dataset with only 6 categories out of the 20 available in the dataset

In [4]:
from sklearn.datasets import fetch_20newsgroups

In [5]:
# choose, for example, 6 categories
categories = [
    'alt.atheism',
    'comp.windows.x',
    'rec.autos',
    'rec.sport.baseball',
    'sci.electronics',
    'sci.space',
]

train = fetch_20newsgroups(subset='train', 
                                categories=categories,
                                remove=('headers', 'footers', 'quotes')
                          )

test = fetch_20newsgroups(subset='test', 
                                categories=categories,
                                remove=('headers', 'footers', 'quotes')
                          )

In [6]:
train_data = pd.DataFrame({'text' : train['data'], 
                           'category' : train['target']})
train_data.head()

,text,category
0,"Benedikt Rosenau writes, with great authority:...",0
1,\nI don't understand this last statement about...,2
2,I'd like to compile X11r5 on a Sony NWS-1750 r...,1
3,"\n\n\nHow do you know it's based on ignorance,...",0
4,\nmuch crap deleted\n\n\nDEAD WRONG! Last tim...,3


In [7]:
test_data = pd.DataFrame({'text' : test['data'], 
                           'category' : test['target']})
test_data.head()

,text,category
0,I can see it now emblazened across the evening...,5
1,The color of the board shows the composition o...,4
2,Regarding the feasability of retrieving the HS...,5
3,\nI believe Acker got a ring from his wife whe...,3
4,\n\nThe new Cruisers DO NOT have independent s...,2


In [8]:
train['target_names']

['alt.atheism',
 'comp.windows.x',
 'rec.autos',
 'rec.sport.baseball',
 'sci.electronics',
 'sci.space']

In [9]:
# topics
train_data.category.value_counts()

3    597
2    594
5    593
1    593
4    591
0    480
Name: category, dtype: int64

In [10]:
# topics
test_data.category.value_counts()

3    397
2    396
1    395
5    394
4    393
0    319
Name: category, dtype: int64

In [49]:
# space
print(train_data[train_data.category==5].sample().iloc[0,0])



I think I can. Largely as a result of efforts by people reading this group
writing letters and making phone calls the following has happened:

1. NASA reprogrammed funds to keep NASP alive in 1991.
2. Efforts to kill DC-X and the SSRT progam where twice twarted
   (Feb. and June of last year).
3. Gouldin kept his job in spite of heavy lobbying against him.

This may not be what Mark was thinking of but it shows that the
readers of sci.space DO have power and influence.

  Allen



**Goal**:  classify documents from the dataset by their topic

## Training a Naive Bayes model

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [26]:
X_train = train_data.text
y_train = train_data.category

In [27]:
pipe = Pipeline(steps=[
    ('vect', TfidfVectorizer()), 
    ('clf', MultinomialNB()) 
])

In [28]:
params_dic =  {'vect__max_features' : [1000,2000,5000,10000],
               'vect__stop_words' : ['english', None],
               'vect__min_df' : [5,10,20,50],
               'vect__ngram_range' : [(1,1), (1,2),(1,3)],
               'vect__use_idf' : [True,False]}

In [29]:
grid = GridSearchCV(pipe,params_dic,scoring='accuracy',cv=5, n_jobs=-1, verbose=True)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  6.0min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__max_features': [1000, 2000, 5000, 10000],
                         'vect__min_df': [5, 10, 20, 50],
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                         'vect__stop_words': ['english', None],
                         'vect__use_idf': [True, False]},
             scoring='accuracy', verbose=True)

In [30]:
grid.best_score_

0.8645653225636819

In [31]:
grid.best_params_

{'vect__max_features': 10000,
 'vect__min_df': 5,
 'vect__ngram_range': (1, 3),
 'vect__stop_words': 'english',
 'vect__use_idf': True}

In [32]:
best_clf = grid.best_estimator_

In [33]:
# evaluate the model
X_test = test_data.text
y_test = test_data.category
y_test_pred = best_clf.predict(X_test)

In [37]:
confusion_matrix(y_test,y_test_pred)

array([[235,   9,  20,  19,   4,  32],
       [  0, 365,   4,  13,   6,   7],
       [  5,   5, 323,  34,  18,  11],
       [  8,  10,   7, 366,   5,   1],
       [  6,  25,  28,  17, 303,  14],
       [ 15,  10,  13,  25,  13, 318]], dtype=int64)

In [38]:
train['target_names']

['alt.atheism',
 'comp.windows.x',
 'rec.autos',
 'rec.sport.baseball',
 'sci.electronics',
 'sci.space']

In [39]:
best_clf.predict(['I always wanted to be an astronaut','I hate Windows 10'])

array([5, 1], dtype=int64)